Для интеграции искусственного интеллекта в систему персонализированных рекомендаций можно построить рекомендательную систему с использованием методов **Collaborative Filtering** (коллаборативная фильтрация) и **Content-based Filtering** (контентно-ориентированная фильтрация). Эти подходы часто используются в рекомендациях, когда данные о пользователях и их предпочтениях могут быть применены для автоматической генерации персонализированных планов тренировок.

### **Подходы для создания рекомендательной системы:**
* **Collaborative Filtering (коллаборативная фильтрация):**  
Основана на анализе похожих пользователей: система предлагает рекомендации на основе того, какие тренировки и активности предпочитали пользователи с похожими паттернами поведения.
* **Content-based Filtering (контентно-ориентированная фильтрация):**  
Основана на предпочтениях самого пользователя: система использует индивидуальные данные (например, уровень активности, ЧСС, сон, шаги) для подбора рекомендаций на основе похожих тренировок, которые он ранее выполнял.

### **Шаги для создания рекомендательной системы:**
1. **Сбор данных о предпочтениях пользователей:**  
Необходимо собрать данные о том, какие тренировки и планы уже использовались пользователями, а также их отзывы о тренировках (например, успешность выполнения, удовлетворенность).
2. **Построение матрицы взаимодействий:**  
На основе данных взаимодействий пользователей с тренировками можно построить матрицу взаимодействий, которая будет использоваться для обучения модели.
3. **Обучение модели:**  
Обучим модель на основе метода коллаборативной фильтрации (например, ALS - Alternating Least Squares) или контентно-ориентированной фильтрации.
4. **Рекомендации на основе предпочтений и похожих пользователей:**  
Модель будет рекомендовать тренировки, которые похожи на те, что были выбраны пользователем ранее, либо на те, что понравились пользователям с похожими предпочтениями.

**Шаг 1: Сбор данных**

In [1]:
import pandas as pd

# Данные о пользователях и их предпочтениях в тренировках (1 - понравилось, 0 - не понравилось)
data = {'UserId': [1, 1, 2, 2, 3, 3, 4, 4],
        'WorkoutId': [101, 102, 101, 103, 102, 103, 101, 104],
        'Rating': [1, 0, 1, 1, 1, 0, 1, 1]}  # Оценки тренировок пользователями
ratings_df = pd.DataFrame(data)
print(ratings_df)


   UserId  WorkoutId  Rating
0       1        101       1
1       1        102       0
2       2        101       1
3       2        103       1
4       3        102       1
5       3        103       0
6       4        101       1
7       4        104       1


**Шаг 2: Коллаборативная фильтрация (ALS)**  
Для создания рекомендаций на основе взаимодействий пользователей с тренировками можно использовать метод **ALS (Alternating Least Squares)**. Он позволит нам найти скрытые факторы, которые могут влиять на предпочтения пользователя.

Использование библиотеки **surprise** для коллаборативной фильтрации:

In [3]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357292 sha256=2c74d0a66fb7a2dc8f1a790cac0873ff7267e673f13dcd16741562456de59d7d
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [4]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Подготовка данных для коллаборативной фильтрации
reader = Reader(rating_scale=(0, 1))  # Рейтинги 0 или 1
data = Dataset.load_from_df(ratings_df[['UserId', 'WorkoutId', 'Rating']], reader)

# Разделяем данные на обучающую и тестовую выборки
trainset, testset = train_test_split(data, test_size=0.2)

# Используем SVD для коллаборативной фильтрации
model = SVD()
model.fit(trainset)

# Прогнозируем на тестовой выборке
predictions = model.test(testset)

# Оценка модели
accuracy.rmse(predictions)



RMSE: 0.3941


0.3940980428505354

**Шаг 3: Генерация рекомендаций для пользователей**  
Теперь, обучив модель, мы можем давать пользователям персонализированные рекомендации по тренировкам на основе их прошлых предпочтений и взаимодействий.

Пример рекомендаций для конкретного пользователя:

In [5]:
# Прогноз для конкретного пользователя
user_id = 1  # Идентификатор пользователя
workout_id = 103  # Идентификатор тренировки

predicted_rating = model.predict(user_id, workout_id)
print(f"Предсказанный рейтинг для пользователя {user_id} на тренировку {workout_id}: {predicted_rating.est}")

# Получение рекомендаций для пользователя
workouts = ratings_df['WorkoutId'].unique()
recommendations = []

for workout in workouts:
    if workout not in ratings_df[ratings_df['UserId'] == user_id]['WorkoutId'].values:
        pred = model.predict(user_id, workout)
        recommendations.append((workout, pred.est))

# Сортировка по наивысшему рейтингу
recommendations.sort(key=lambda x: x[1], reverse=True)

# Вывод топ-3 тренировок для пользователя
top_recommendations = recommendations[:3]
print("Рекомендуемые тренировки для пользователя:")
for workout, rating in top_recommendations:
    print(f"Тренировка {workout}, предсказанный рейтинг: {rating:.2f}")


Предсказанный рейтинг для пользователя 1 на тренировку 103: 0.5596659505715381
Рекомендуемые тренировки для пользователя:
Тренировка 104, предсказанный рейтинг: 0.62
Тренировка 103, предсказанный рейтинг: 0.56


**Шаг 4: Контентно-ориентированная фильтрация**
Мы также можем создать контентно-ориентированную фильтрацию, которая будет учитывать данные о тренировках (например, типы тренировок, интенсивность, количество шагов) и подбирать наиболее подходящие на основе предпочтений пользователя.

Пример реализации:

In [6]:
# Пример данных о тренировках (WorkoutId, тип тренировки, интенсивность)
workouts_data = {
    'WorkoutId': [101, 102, 103, 104],
    'WorkoutType': ['Cardio', 'Strength', 'Flexibility', 'Endurance'],
    'Intensity': [3, 4, 2, 5]  # Интенсивность тренировки
}
workouts_df = pd.DataFrame(workouts_data)

# Данные о предпочтениях пользователя
user_preferences = {
    'PreferredType': 'Cardio',  # Предпочитаемый тип тренировки
    'PreferredIntensity': 3     # Предпочитаемая интенсивность
}

# Фильтрация тренировок на основе предпочтений пользователя
filtered_workouts = workouts_df[
    (workouts_df['WorkoutType'] == user_preferences['PreferredType']) &
    (workouts_df['Intensity'] == user_preferences['PreferredIntensity'])
]

print("Рекомендованные тренировки на основе контентной фильтрации:")
print(filtered_workouts)


Рекомендованные тренировки на основе контентной фильтрации:
   WorkoutId WorkoutType  Intensity
0        101      Cardio          3


**Шаг 5: Интеграция рекомендаций в систему**
После создания модели рекомендаций (как коллаборативной, так и контентно-ориентированной) можно интегрировать её в систему:

* **Объединение с уже созданной системой адаптивных тренировок:**   
  Рекомендации могут быть частью более общей системы, где пользователь получает как персонализированный план тренировок, так и рекомендации по типам тренировок на основе своих предпочтений.

* **Автоматическое обновление предпочтений:**   
  Система может обучаться в реальном времени на данных взаимодействий пользователя с тренировками (например, сколько раз пользователь выполнял определённые тренировки и насколько они ему понравились).

* **Динамическая адаптация рекомендаций:**   
  Рекомендации могут автоматически обновляться в зависимости от изменений в предпочтениях пользователя (например, при изменении целей или уровня активности).